In [3]:
import astropy.units as u
import astropy.constants as c
import numpy as np
import matplotlib.pyplot as plt
from astroquery.gaia import Gaia

In [33]:
# Define and execute the ADQL query to obtain Gaia DR3 targets
query = f"""
SELECT *, (DISTANCE(132.825, 11.8, ra, dec)) AS ang_sep
FROM gaiadr3.gaia_source
WHERE DISTANCE(132.825, 11.8, ra, dec) < 1.
AND phot_g_mean_mag < 13
AND parallax IS NOT NULL
ORDER BY ang_sep ASC
"""

# Upload the source_id table for crossmatching
job = Gaia.launch_job_async(query=query)
gaia = job.get_results()

INFO: Query finished. [astroquery.utils.tap.core]


In [35]:
# Define and execute the ADQL query to crossmatch 2MASS with Gaia DR3
query = f"""
SELECT *, (DISTANCE(132.825, 11.8, ra, dec)) AS ang_sep
FROM gaiadr1.tmass_original_valid
WHERE DISTANCE(132.825, 11.8, ra, dec) < 1.
ORDER BY ang_sep ASC
"""

# Upload the source_id table for crossmatching
job = Gaia.launch_job_async(query=query)
tmass = job.get_results()

INFO: Query finished. [astroquery.utils.tap.core]


In [36]:
# Define and execute the ADQL query to crossmatch 2MASS with Gaia DR3
query = f"""
SELECT *, (DISTANCE(132.825, 11.8, ra, dec)) AS ang_sep
FROM gaiadr3.gaia_source
JOIN gaiaedr3.tmass_psc_xsc_best_neighbour AS xmatch USING (source_id)
JOIN gaiaedr3.tmass_psc_xsc_join AS xjoin
   ON xmatch.original_ext_source_id = xjoin.original_psc_source_id
JOIN gaiadr1.tmass_original_valid AS tmass
   ON xjoin.original_psc_source_id = tmass.designation
WHERE DISTANCE(132.825, 11.8, ra, dec) < 1.
ORDER BY ang_sep ASC
"""

# Upload the source_id table for crossmatching
job = Gaia.launch_job_async(query=query)
tmass = job.get_results()

500 Error 500:
null


HTTPError: Error 500:
null